In [ ]:
pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [3]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [5]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from langchain.document_loaders import TextLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import streamlit as st
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
from huggingface_hub import login
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.vectorstores import Qdrant
from transformers import AutoModelForSeq2SeqLM

In [2]:
csv_path = "/content/sample_data/rag-energy-real/Plant_1_Generation_Data.csv"
sum_path = "/content/sample_data/rag-energy-real/solar_summary.txt"
trend_path = "/content/sample_data/rag-energy-real/solar_trend_summary.txt"
policy_pdf_path = "/content/sample_data/rag-energy-real/1698657395807-Gujarat-Renewable-Energy-Policy.pdf"
policy_txt_path = "/content/sample_data/rag-energy-real/Gujarat_RE_Policy_2023.txt"

In [3]:
# preprocess data
plant_df = pd.read_csv(
    csv_path,
    parse_dates=['DATE_TIME'],
    dayfirst=True
)
plant_df['DATE'] = plant_df['DATE_TIME'].dt.date

In [4]:
# aggregate of daily energy generation
daily_df = plant_df.groupby('DATE').agg({
    'DC_POWER': 'sum',
    'AC_POWER': 'sum'
}).reset_index()
daily_df['Efficiency'] = daily_df['AC_POWER'] / daily_df['DC_POWER']

In [5]:
# detect AC_POWER dips
avg_output = daily_df['AC_POWER'].mean()
pattern_summary = []
for _, row in daily_df.iterrows():
    drop = ((avg_output - row['AC_POWER']) / avg_output) * 100
    if drop > 15:
        pattern_summary.append(
            f"On {row['DATE']}, AC power output dropped by {drop:.1f}% with DC power of {row['DC_POWER']:.1f} units."
        )

In [6]:
with open(sum_path, "w") as f:
    for line in pattern_summary:
        f.write(line + "\n")

In [7]:
#trends
summaries = []
X = np.arange(len(daily_df)).reshape(-1, 1)
y = daily_df['AC_POWER'].values

In [8]:
# 1. downward trend
daily_df['DATE'] = pd.to_datetime(daily_df['DATE'])
lr_model = LinearRegression().fit(X, y)
slope = lr_model.coef_[0]
if slope < -50:
    summaries.append(f"Downward trend detected in AC output (slope: {slope:.1f} units/day). Possible efficiency loss or degradation.")

In [9]:
# 2. seasonal variation
if len(daily_df) >= 21:
    weekly_df = daily_df.set_index('DATE').resample('W').mean()
    seasonal_strength = weekly_df['AC_POWER'].std()
    if seasonal_strength > 200:
        summaries.append("Weekly generation shows strong seasonal variation. Monitor environmental impacts.")

In [10]:
# 3. o/p vs irradiance efficiency drop
low_eff_days = daily_df[daily_df['Efficiency'] < daily_df['Efficiency'].quantile(0.1)]
if not low_eff_days.empty:
    summaries.append(f"{len(low_eff_days)} days showed low conversion efficiency. Cleaning or technical checks recommended.")

In [11]:
# 4. predictive trend (short-term slope)
forecast_model = LinearRegression().fit(X[-30:], y[-30:])
forecast_slope = forecast_model.coef_[0]
if forecast_slope < -50:
    summaries.append("Short-term forecast suggests declining output. Maintenance or shading issues may be present.")

# 5. o/p volatility under stable irradiance
if 'DC_POWER' in plant_df.columns:
    stable_irr = daily_df[(daily_df['Efficiency'] > 0.95) & (daily_df['Efficiency'] < 1.05)]
    if not stable_irr.empty:
        volatility = stable_irr['AC_POWER'].std()
        if volatility > 150:
            summaries.append(f"High output variance ({volatility:.1f} units) under stable efficiency. May indicate inverter or connection instability.")

In [12]:
with open(trend_path, "w") as f:
    for line in summaries:
        f.write(line + "\n")

In [13]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
login(os.environ["HUGGINGFACEHUB_API_TOKEN"])

In [28]:
pip install qdrant-client langchain sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [21]:
pip install -U langchain-huggingface

In [19]:
# vectordb
sum_loader = TextLoader("/content/sample_data/rag-energy-real/solar_summary.txt")
trend_loader = TextLoader("/content/sample_data/rag-energy-real/solar_trend_summary.txt")
policy_loader = TextLoader("/content/sample_data/rag-energy-real/Gujarat_RE_Policy_2023.txt")
docs = sum_loader.load() + trend_loader.load() + policy_loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

qdrant_client = QdrantClient(
    url="",
    api_key=""
)

collection_name = "rag_energy_cloud"

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=embedder,
)

vectorstore.add_documents(split_docs)
retriever = vectorstore.as_retriever()

<ipython-input-19-2a2360b990a7>:37: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


In [22]:
# llm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=pipe)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [26]:
def format_answer(raw_text: str) -> str:
    lines = raw_text.splitlines()
    summary = lines[0] if lines else "No summary found."
    drops = [line for line in lines if "AC power output dropped" in line]
    forecast = next((line for line in lines if "forecast" in line.lower()), "No forecast trend found.")

    return f"""
### Summary Answer:
{summary}

### Recommended Actions:
- Clean the panels
- Inspect the inverter
- Monitor shading or degradation

"""

In [27]:
query = "Why did AC output drop significantly on certain days?"
result = qa_chain.invoke(query)
raw_answer = result['result']

print(format_answer(raw_answer))

print("\n### Sources:")
for doc in result['source_documents']:
    print("-", doc.metadata.get('source', 'Unknown'))


### Summary Answer:
Low conversion efficiency. Cleaning or technical checks recommended. Short-term forecast suggests declining output. Maintenance or shading issues may be present. On 2020-05-18, AC power output dropped by 24.4% with DC power of 4812548.5 units. On 2020-05-20, AC power output dropped by 15.7% with DC power of 5375585.8 units. On 2020-05-31, AC power output dropped by 15.7% with DC power of 5375585.8 units. On 2020-06-01, AC power output dropped by 18.1% with DC power of 5210134.0 units. On 2020-06-11, AC power output dropped by 20.6% with DC power of 5045678.6 units. On 2020-06-12, AC power output dropped by 17.9% with DC power of 5222353.9 units. On 2020-06-17, AC power output dropped by 16.4% with DC power of 5317779.8 units.

### Recommended Actions:
- Clean the panels
- Inspect the inverter
- Monitor shading or degradation



### Sources:
- /content/sample_data/rag-energy-real/solar_trend_summary.txt
- /content/sample_data/rag-energy-real/solar_summary.txt
- /con

In [28]:
!pip install pyngrok

In [60]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

sum_loader = TextLoader("/content/sample_data/rag-energy-real/solar_summary.txt")
trend_loader = TextLoader("/content/sample_data/rag-energy-real/solar_trend_summary.txt")
policy_loader = TextLoader("/content/sample_data/rag-energy-real/Gujarat_RE_Policy_2023.txt")
docs = sum_loader.load() + trend_loader.load() + policy_loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# qdrant
qdrant_client = QdrantClient(
    url="",
    api_key=""

collection_name = "rag_energy_cloud"

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=embedder,
)

vectorstore.add_documents(split_docs)
retriever = vectorstore.as_retriever()

# llm
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=pipe)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

def format_answer(raw_text: str) -> str:
    lines = raw_text.splitlines()
    summary = lines[0] if lines else "No summary found."
    drops = [line for line in lines if "AC power output dropped" in line]
    forecast = next((line for line in lines if "forecast" in line.lower()), "No forecast trend found.")
    return f"""
### Summary Answer:
{summary}

### Recommended Actions:
- Clean the panels
- Inspect the inverter
- Monitor shading or degradation

"""

# streamlit
st.set_page_config(page_title="Renewable Energy Assistant", page_icon="🔋")
st.title("Renewable Energy Assistant for Solar energy")

query = st.text_input(
    "Ask a question about solar output or any policy:",
    "Why did AC output drop significantly on certain days?"
)

if st.button("Get Answer"):
    result = qa_chain.invoke(query)
    raw_answer = result['result']

    st.subheader("Answer")
    st.markdown(format_answer(raw_answer))

    st.subheader("Sources")
    for doc in result['source_documents']:
        st.markdown(f"- `{doc.metadata.get('source', 'Unknown')}`")


Overwriting app.py


In [61]:
from pyngrok import conf
conf.get_default().auth_token = ""

In [62]:
!pkill -f ngrok

In [63]:
from pyngrok import ngrok
import time

!streamlit run app.py &>/content/logs.txt &

time.sleep(5)
public_url = ngrok.connect(addr=8501, proto="http")

print("Streamlit App URL:", public_url)

Streamlit App URL: NgrokTunnel: "https://123e-35-201-248-114.ngrok-free.app" -> "http://localhost:8501"
